In [4]:
import os


def get_log_files(directory):
    """
    Retrieves all log files in the specified directory.

    :param directory: Path to the directory to search for log files.
    :return: List of paths to log files found in the directory.
    """
    log_files = [os.path.join(directory, file) for file in os.listdir(directory) if file.endswith('.log')]
    return log_files


# Example usage
directory_path = os.path.join('ed_02', 'r1')  # Replace with the actual directory path
log_files = get_log_files(directory_path)
print(log_files)


# Function to parse the log file and extract only WARNING log entries
def extract_warning_logs(input_path, output_path):
    with open(input_path, 'r') as input_file, open(output_path, 'w') as output_file:
        for line in input_file:
            if "WARNING" in line:
                output_file.write(line)

for log_file in log_files:
    # Call the function
    extract_warning_logs(log_file, log_file.replace('.log', 'defcon_warn.log'))


['ed_02\\r1\\20240208_055445_192.168.68.50_viability_eval.log', 'ed_02\\r1\\20240208_055449_192.168.68.70_viability_eval.log', 'ed_02\\r1\\20240208_055453_192.168.68.60_viability_eval.log', 'ed_02\\r1\\20240208_055516_192.168.68.61_viability_eval.log', 'ed_02\\r1\\20240208_055622_192.168.68.71_viability_eval.log', 'ed_02\\r1\\20240208_055720_192.168.68.73_viability_eval_trigger.log']
